In [1]:
!pip install tensorflow==2.11.0 -q

In [2]:
# !git clone https://github.com/jeroencvlier/utils.git
!pip install -U -i https://test.pypi.org/simple/ gym-OT

Looking in indexes: https://test.pypi.org/simple/


In [3]:
import gym, os, glob
from tqdm import tqdm
from gym_OT.envs.OTGym_env import OTEnv_v0
from ray import tune
import gym_OT

select_env = "OT-v0"
tune.register_env(select_env, lambda: OTEnv_v0({}))

import ray
from ray.rllib.algorithms.dqn import DQN
# from ray.rllib.algorithms.ppo import PPO
# from ray.rllib.algorithms.sac import SAC

from logger_ import custom_log_creator 

ray.shutdown()
ray.init(ignore_reinit_error=True,num_cpus=8,num_gpus=1)

/usr/local/lib/python3.8/dist-packages/marshmallow/__init__.py:19: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  __version_info__ = tuple(LooseVersion(__version__).version)
2022-11-23 12:55:43,404	INFO worker.py:1528 -- Started a local Ray instance.


Python version:,3.8.10
Ray version:,2.1.0


In [4]:
import shutil,glob,os
# [shutil.rmtree(x) for x in glob.glob('/notebooks/DQN_Checkpoint/*/')]
# [os.remove(x) for x in glob.glob('/notebooks/DQN_Checkpoint/*.*')]

replay_buffer_config = {
            "type": "MultiAgentPrioritizedReplayBuffer",
            # Specify prioritized replay by supplying a buffer type that supports
            # prioritization, for example: MultiAgentPrioritizedReplayBuffer.
            "prioritized_replay": DEPRECATED_VALUE,
            # Size of the replay buffer. Note that if async_updates is set,
            # then each worker will have a replay buffer of this size.
            "capacity": 50000,
            "prioritized_replay_alpha": 0.6,
            # Beta parameter for sampling from prioritized replay buffer.
            "prioritized_replay_beta": 0.4,
            # Epsilon to add to the TD errors when updating priorities.
            "prioritized_replay_eps": 1e-6,
            # The number of continuous environment steps to replay at once. This may
            # be set to greater than 1 to support recurrent models.
            "replay_sequence_length": 1,
            # Whether to compute priorities on workers.
            "worker_side_prioritization": False,
        }

In [ ]:
# Configure the algorithm.
config = {
    "num_workers": 7,
    "num_gpus_per_worker":0.125,
    "num_envs_per_worker":1,
    "num_gpus": 1,
    "framework": "torch",
    "model": {"fcnet_hiddens": [800, 800, 400],
              "fcnet_activation": "relu"},
    "evaluation_num_workers": 1,
    "evaluation_config": {"render_env": False,
    },
    "replay_buffer_config":{"capacity": 500000}
    
}

algo = DQN(env=OTEnv_v0,config=config,logger_creator=custom_log_creator("/notebooks/ray_results", 'DQN'))

# algo = PPO(env=OTEnv_v0,config=config,logger_creator=custom_log_creator("/notebooks/ray_results", 'PPO'))
# algo = SAC(env=OTEnv_v0,config=config,logger_creator=custom_log_creator("/notebooks/ray_results", 'SAC'))

if len(glob.glob('/notebooks/DQN_Checkpoint/checkpoint_*'))>1:
    print('Restoring checkpoint: ',sorted(glob.glob('/notebooks/DQN_Checkpoint/checkpoint_*'))[-2])
    algo.restore('/notebooks/'+sorted(glob.glob('DQN_Checkpoint/*'))[-2])
    shutil.rmtree(sorted(glob.glob('/notebooks/DQN_Checkpoint/checkpoint_*'))[-1])
    
for _ in tqdm(range(100000)):
    for _ in range(25):
        algo.train()
    checkpoint = algo.save("/notebooks/DQN_Checkpoint")
    [shutil.rmtree(x) for x in  sorted(glob.glob('/notebooks/DQN_Checkpoint/*/'))[:-50]]

algo.evaluate()


2022-11-23 12:55:44,913	WARNING algorithm.py:2531 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
2022-11-23 12:55:44,916	INFO simple_q.py:307 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting `simple_optimizer=True` if this doesn't work for you.
2022-11-23 12:55:44,918	INFO algorithm.py:457 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=506) /usr/local/lib/python3.8/dist-packages/marshmallow/__init__.py:19: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(RolloutWorker pid=506)   __version_info__ = tuple(LooseVersion(__version__).version)
(RolloutWorker pid=507) /usr/local/

(RolloutWorker pid=506) mounted drive detected!
(RolloutWorker pid=506) Loading Scalers
(RolloutWorker pid=507) mounted drive detected!
(RolloutWorker pid=507) Loading Scalers
(RolloutWorker pid=505) mounted drive detected!
(RolloutWorker pid=505) Loading Scalers
(RolloutWorker pid=515) mounted drive detected!
(RolloutWorker pid=515) Loading Scalers
(RolloutWorker pid=511) mounted drive detected!
(RolloutWorker pid=511) Loading Scalers


(RolloutWorker pid=513) /usr/local/lib/python3.8/dist-packages/marshmallow/__init__.py:19: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(RolloutWorker pid=513)   __version_info__ = tuple(LooseVersion(__version__).version)
(RolloutWorker pid=505) 2022-11-23 12:55:50,541	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


(RolloutWorker pid=509) mounted drive detected!
(RolloutWorker pid=509) Loading Scalers
(RolloutWorker pid=513) mounted drive detected!
(RolloutWorker pid=513) Loading Scalers


2022-11-23 12:55:57,818	WARNING deprecation.py:47 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
2022-11-23 12:55:57,819	WARNING algorithm.py:2531 -- You have specified 1 evaluation workers, but your `evaluation_interval` is None! Therefore, evaluation will not occur automatically with each call to `Algorithm.train()`. Instead, you will have to call `Algorithm.evaluate()` manually in order to trigger an evaluation run.
(RolloutWorker pid=746) /usr/local/lib/python3.8/dist-packages/marshmallow/__init__.py:19: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
(RolloutWorker pid=746)   __version_info__ = tuple(LooseVersion(__version__).version)
(RolloutWorker pid=746) 2022-11-23 12:56:01,894	WARNING env.py:147 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' wi

(RolloutWorker pid=746) mounted drive detected!
(RolloutWorker pid=746) Loading Scalers


2022-11-23 12:56:04,462	INFO trainable.py:164 -- Trainable.setup took 19.550 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-11-23 12:56:04,649	INFO trainable.py:766 -- Restored on 10.42.20.184 from checkpoint: /notebooks/DQN_Checkpoint/checkpoint_020000
2022-11-23 12:56:04,650	INFO trainable.py:775 -- Current state after restoring: {'_iteration': 20000, '_timesteps_total': None, '_time_total': 68345.74181818962, '_episodes_total': 247591}


Restoring checkpoint:  /notebooks/DQN_Checkpoint/checkpoint_020000


  0%|          | 0/100000 [00:00<?, ?it/s]2022-11-23 12:56:06,980	WARNING multi_agent_prioritized_replay_buffer.py:215 -- Adding batches with column `weights` to this buffer while providing weights as a call argument to the add method results in the column being overwritten.
2022-11-23 12:56:06,985	WARNING deprecation.py:47 -- DeprecationWarning: `concat_samples` has been deprecated. Use `concat_samples() from rllib.policy.sample_batch` instead. This will raise an error in the future!
  0%|          | 31/100000 [38:26<2222:29:29, 80.03s/it]

In [ ]:
# config = DQNConfig()

# config = config.resources(num_gpus=1,num_gpus_per_worker = 0.125) 
# config = config.rollouts(num_rollout_workers=4,num_envs_per_worker=2)
# config = config.environment(OTEnv_v0)
# config = config.framework("torch")

# # network
# config.model['fcnet_hiddens'] = ['256','256']
# config.model['fcnet_activation'] = "relu"

# # tunning
# config = config.training(lr=tune.grid_search([0.005, 0.003, 0.001, 0.0001]))

# # eval
# config.evaluation_num_workers = 1